In [18]:
import pandas as pd
import numpy as np
import xgboost as xgb

from copy import copy
import datetime
import pickle
from geopy.geocoders import Nominatim

# Open Saved Model

In [14]:
filename = "xgb_model.sav"

In [15]:
loaded_model = pickle.load(open(filename, 'rb'))

# Genetic Algorithm

In [19]:
#Sample date

date_list = [4, 6, 2016] #April 6, 2016

year = int(date_list[2])
month = int(date_list[1])
day = int(date_list[0])

my_date = datetime.date(year, month, day)

In [4]:
#Sample test locations

test_locations = {'L1': (40.819688, -73.915091),
                  'L2': (40.815421, -73.941761),
                  'L3': (40.764198, -73.910785),
                  'L4': (40.768790, -73.953285),
                  'L5': (40.734851, -73.952950),
                  'L6': (40.743613, -73.977998),
                  'L7': (40.745313, -73.993793),
                  'L8': (40.662713, -73.946101),
                  'L9': (40.703761, -73.886496),
                  'L10': (40.713620, -73.943076),
                  'L11': (40.725212, -73.809179)
             }

In [7]:
geolocator = Nominatim()
addresses = []

for key in test_locations:
    location = geolocator.reverse(test_locations[key])
    addresses.append(location.address)

In [8]:
addresses

['424, East 155th Street, Melrose, Bronx County, NYC, New York, 10455, United States of America',
 '137, West 136th Street, Harlem, Manhattan Community Board 10, New York County, NYC, New York, 10030, United States of America',
 '43-11, 28th Avenue, Astoria, Queens County, NYC, New York, 11103, United States of America',
 '435, East 74th Street, Upper East Side, Manhattan Community Board 8, New York County, NYC, New York, 10021, United States of America',
 '211, Freeman Street, Greenpoint, Kings County, NYC, New York, 11222, United States of America',
 '232, East 32nd Street, Murray Hill, Manhattan Community Board 6, New York County, NYC, New York, 10016, United States of America',
 '159, West 25th Street, Penn Station South, Manhattan Community Board 4, New York County, NYC, New York, 10001, United States of America',
 '486, Brooklyn Avenue, Crown Heights, BK, Kings County, NYC, New York, 11225, United States of America',
 '70-38, 67th Place, Glendale, Queens County, NYC, New York, 11

In [9]:
test_addresses = {'L1': '424 East 155th Street NY',
                  'L2': '137 West 136th Street NY',
                  'L3': '43-11 28th Avenue NY',
                  'L4': '435 East 74th Street NY',
                  'L5': '211 Freeman Street NY',
                  'L6': '232 East 32nd Street NY',
                  'L7': '159 West 25th Street NY',
                  'L8': '486 Brooklyn Avenue NY',
                  'L9': '70-38 67th Place NY',
                  'L10': '194 Devoe Street NY',
                  'L11': '158-46 76th Avenue NY'
             }

# The following code was borrowed from 
##### https://github.com/ZWMiller/PythonProjects/blob/master/genetic_algorithms/evolutionary_algorithm_traveling_salesman.ipynb
# and modified to be applied to my problem

In [16]:
def create_guess(points):
    """
    Creates a possible path between all points, returning to the original.
    Input: List of point IDs
    """
    guess = copy(points)
    np.random.shuffle(guess)
    guess.append(guess[0])
    return list(guess)

create_guess(list(test_locations.keys()))

['L11', 'L2', 'L7', 'L1', 'L4', 'L8', 'L3', 'L9', 'L5', 'L10', 'L6', 'L11']

In [17]:
def create_generation(points, population=100):
    """
    Makes a list of guessed point orders given a list of point IDs.
    Input:
    points: list of point ids
    population: how many guesses to make
    """
    generation = [create_guess(points) for _ in range(population)]
    return generation

test_generation = create_generation(list(test_locations.keys()), population=10)
print(test_generation)

[['L1', 'L5', 'L8', 'L7', 'L11', 'L2', 'L6', 'L10', 'L4', 'L3', 'L9', 'L1'], ['L5', 'L6', 'L2', 'L8', 'L9', 'L7', 'L11', 'L10', 'L1', 'L4', 'L3', 'L5'], ['L11', 'L10', 'L5', 'L3', 'L9', 'L4', 'L2', 'L1', 'L6', 'L7', 'L8', 'L11'], ['L8', 'L9', 'L3', 'L7', 'L10', 'L2', 'L11', 'L5', 'L4', 'L6', 'L1', 'L8'], ['L5', 'L3', 'L11', 'L6', 'L9', 'L10', 'L4', 'L7', 'L8', 'L1', 'L2', 'L5'], ['L8', 'L9', 'L11', 'L2', 'L7', 'L1', 'L5', 'L3', 'L6', 'L4', 'L10', 'L8'], ['L6', 'L10', 'L9', 'L2', 'L5', 'L1', 'L8', 'L4', 'L3', 'L11', 'L7', 'L6'], ['L2', 'L9', 'L8', 'L4', 'L6', 'L1', 'L5', 'L10', 'L3', 'L7', 'L11', 'L2'], ['L2', 'L4', 'L9', 'L7', 'L10', 'L11', 'L3', 'L5', 'L1', 'L6', 'L8', 'L2'], ['L4', 'L3', 'L6', 'L5', 'L10', 'L8', 'L7', 'L11', 'L2', 'L9', 'L1', 'L4']]


In [24]:
def travel_time_between_points(point1_id, point2_id, hour, date, passenger_count = 1, 
                               store_and_fwd_flag = 0, pickup_minute = 0):
    """
    Given two points, this calculates travel between them based on a XGBoost predictive model
    """
    
    model_data = {'passenger_count': passenger_count,
                  'pickup_longitude' : point1_id[1],
                  'pickup_latitude' : point1_id[0],
                  'dropoff_longitude' : point2_id[1],
                  'dropoff_latitude' : point2_id[0],
                  'store_and_fwd_flag' : store_and_fwd_flag,
                  'pickup_month' : my_date.month,
                  'pickup_day' : my_date.day,
                  'pickup_weekday' : my_date.weekday(),
                  'pickup_hour': hour,
                  'pickup_minute' : pickup_minute,
                  'latitude_difference' : point2_id[0] - point1_id[0],
                  'longitude_difference' : point2_id[1] - point1_id[1],
                  'trip_distance' : 0.621371 * 6371 * (abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(point2_id[0] - point1_id[0]) * np.pi / 180) / 2))), 
                                  np.sqrt(1-(np.square(np.sin((abs(point2_id[0] - point1_id[0]) * np.pi / 180) / 2)))))) + \
                                     abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(point2_id[1] - point1_id[1]) * np.pi / 180) / 2))), 
                                  np.sqrt(1-(np.square(np.sin((abs(point2_id[1] - point1_id[1]) * np.pi / 180) / 2)))))))
                 }

    df = pd.DataFrame([model_data], columns=model_data.keys())
    
    pred = np.exp(loaded_model.predict(xgb.DMatrix(df))) - 1
    
    return pred[0]

In [25]:
coordinates = test_locations

In [26]:
def fitness_score(guess):
    """
    Loops through the points in the guesses order and calculates
    how much distance the path would take to complete a loop.
    Lower is better.
    """
    score = 0
    for ix, point_id in enumerate(guess[:-1]):
        score += travel_time_between_points(coordinates[point_id], coordinates[guess[ix+1]], 11, my_date)
    return score

def check_fitness(guesses):
    """
    Goes through every guess and calculates the fitness score. 
    Returns a list of tuples: (guess, fitness_score)
    """
    fitness_indicator = []
    for guess in guesses:
        fitness_indicator.append((guess, fitness_score(guess)))
    return fitness_indicator

print(check_fitness(test_generation))

[(['L1', 'L5', 'L8', 'L7', 'L11', 'L2', 'L6', 'L10', 'L4', 'L3', 'L9', 'L1'], 292.88662338256836), (['L5', 'L6', 'L2', 'L8', 'L9', 'L7', 'L11', 'L10', 'L1', 'L4', 'L3', 'L5'], 272.61923122406006), (['L11', 'L10', 'L5', 'L3', 'L9', 'L4', 'L2', 'L1', 'L6', 'L7', 'L8', 'L11'], 230.00274276733398), (['L8', 'L9', 'L3', 'L7', 'L10', 'L2', 'L11', 'L5', 'L4', 'L6', 'L1', 'L8'], 283.04906272888184), (['L5', 'L3', 'L11', 'L6', 'L9', 'L10', 'L4', 'L7', 'L8', 'L1', 'L2', 'L5'], 276.35141468048096), (['L8', 'L9', 'L11', 'L2', 'L7', 'L1', 'L5', 'L3', 'L6', 'L4', 'L10', 'L8'], 246.69790363311768), (['L6', 'L10', 'L9', 'L2', 'L5', 'L1', 'L8', 'L4', 'L3', 'L11', 'L7', 'L6'], 274.2643165588379), (['L2', 'L9', 'L8', 'L4', 'L6', 'L1', 'L5', 'L10', 'L3', 'L7', 'L11', 'L2'], 275.72143936157227), (['L2', 'L4', 'L9', 'L7', 'L10', 'L11', 'L3', 'L5', 'L1', 'L6', 'L8', 'L2'], 285.8990898132324), (['L4', 'L3', 'L6', 'L5', 'L10', 'L8', 'L7', 'L11', 'L2', 'L9', 'L1', 'L4'], 276.7117729187012)]


In [27]:
def get_breeders_from_generation(guesses, take_best_N=10, take_random_N=5, verbose=False, mutation_rate=0.1):
    """
    This sets up the breeding group for the next generation. You have
    to be very careful how many breeders you take, otherwise your
    population can explode. These two, plus the "number of children per couple"
    in the make_children function must be tuned to avoid exponential growth or decline!
    """
    # First, get the top guesses from last time
    fit_scores = check_fitness(guesses)
    sorted_guesses = sorted(fit_scores, key=lambda x: x[1]) # sorts so lowest is first, which we want
    new_generation = [x[0] for x in sorted_guesses[:take_best_N]]
    best_guess = new_generation[0]
    
    if verbose:
        # If we want to see what the best current guess is!
        print(best_guess)
    
    # Second, get some random ones for genetic diversity
    for _ in range(take_random_N):
        ix = np.random.randint(len(guesses))
        new_generation.append(guesses[ix])
        
    # No mutations here since the order really matters.
    # If we wanted to, we could add a "swapping" mutation,
    # but in practice it doesn't seem to be necessary
    
    np.random.shuffle(new_generation)
    return new_generation, best_guess

def make_child(parent1, parent2):
    """ 
    Take some values from parent 1 and hold them in place, then merge in values
    from parent2, filling in from left to right with cities that aren't already in 
    the child. 
    """
    list_of_ids_for_parent1 = list(np.random.choice(parent1, replace=False, size=len(parent1)//2))
    child = [-99 for _ in parent1]
    
    for ix in range(0, len(list_of_ids_for_parent1)):
        child[ix] = parent1[ix]
    for ix, gene in enumerate(child):
        if gene == -99:
            for gene2 in parent2:
                if gene2 not in child:
                    child[ix] = gene2
                    break
    child[-1] = child[0]
    return child

def make_children(old_generation, children_per_couple=1):
    """
    Pairs parents together, and makes children for each pair. 
    If there are an odd number of parent possibilities, one 
    will be left out. 
    
    Pairing happens by pairing the first and last entries. 
    Then the second and second from last, and so on.
    """
    mid_point = len(old_generation)//2
    next_generation = [] 
    
    for ix, parent in enumerate(old_generation[:mid_point]):
        for _ in range(children_per_couple):
            next_generation.append(make_child(parent, old_generation[-ix-1]))
    return next_generation

In [28]:
current_generation = create_generation(list(test_locations.keys()),population=500)
print_every_n_generations = 5

for i in range(100):
    if not i % print_every_n_generations:
        print("Generation %i: "%i, end='')
        print(len(current_generation))
        is_verbose = True
    else:
        is_verbose = False
    breeders, best_guess = get_breeders_from_generation(current_generation, 
                                                        take_best_N=250, take_random_N=100, 
                                                        verbose=is_verbose)
    current_generation = make_children(breeders, children_per_couple=3)

Generation 0: 500
['L10', 'L8', 'L7', 'L2', 'L1', 'L3', 'L11', 'L9', 'L6', 'L4', 'L5', 'L10']
Generation 5: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 10: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 15: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 20: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 25: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 30: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 35: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 40: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 45: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11', 'L3', 'L2', 'L1', 'L4', 'L5']
Generation 50: 525
['L5', 'L6', 'L7', 'L10', 'L8', 'L9', 'L11

In [29]:
def evolve_to_solve(current_generation, max_generations, take_best_N, take_random_N,
                    mutation_rate, children_per_couple, print_every_n_generations, verbose=False):
    """
    Takes in a generation of guesses then evolves them over time using our breeding rules.
    Continue this for "max_generations" times.
    Inputs:
    current_generation: The first generation of guesses
    max_generations: how many generations to complete
    take_best_N: how many of the top performers get selected to breed
    take_random_N: how many random guesses get brought in to keep genetic diversity
    mutation_rate: How often to mutate (currently unused)
    children_per_couple: how many children per breeding pair
    print_every_n_geneartions: how often to print in verbose mode
    verbose: Show printouts of progress
    Returns:
    fitness_tracking: a list of the fitness score at each generations
    best_guess: the best_guess at the end of evolution
    """
    fitness_tracking = []
    for i in range(max_generations):
        if verbose and not i % print_every_n_generations and i > 0:
            print("Generation %i: "%i, end='')
            print(len(current_generation))
            print("Current Best Score: ", fitness_tracking[-1])
            is_verbose = True
        else:
            is_verbose = False
        breeders, best_guess = get_breeders_from_generation(current_generation, 
                                                            take_best_N=take_best_N, take_random_N=take_random_N, 
                                                            verbose=is_verbose, mutation_rate=mutation_rate)
        fitness_tracking.append(fitness_score(best_guess))
        current_generation = make_children(breeders, children_per_couple=children_per_couple)
    
    return fitness_tracking, best_guess

current_generation = create_generation(list(test_locations.keys()),population=500)
fitness_tracking, best_guess = evolve_to_solve(current_generation, 100, 150, 70, 0.5, 3, 5, verbose=True)

Generation 5: 330
Current Best Score:  209.98634338378906
['L11', 'L9', 'L7', 'L6', 'L10', 'L8', 'L5', 'L4', 'L1', 'L2', 'L3', 'L11']
Generation 10: 330
Current Best Score:  209.98634338378906
['L11', 'L9', 'L7', 'L6', 'L10', 'L8', 'L5', 'L4', 'L1', 'L2', 'L3', 'L11']
Generation 15: 330
Current Best Score:  209.98634338378906
['L11', 'L9', 'L7', 'L6', 'L10', 'L8', 'L5', 'L4', 'L1', 'L2', 'L3', 'L11']
Generation 20: 330
Current Best Score:  209.98634338378906
['L11', 'L9', 'L7', 'L6', 'L10', 'L8', 'L5', 'L4', 'L1', 'L2', 'L3', 'L11']
Generation 25: 330
Current Best Score:  209.98634338378906
['L11', 'L9', 'L7', 'L6', 'L10', 'L8', 'L5', 'L4', 'L1', 'L2', 'L3', 'L11']
Generation 30: 330
Current Best Score:  209.98634338378906
['L11', 'L9', 'L7', 'L6', 'L10', 'L8', 'L5', 'L4', 'L1', 'L2', 'L3', 'L11']
Generation 35: 330
Current Best Score:  209.98634338378906
['L11', 'L9', 'L7', 'L6', 'L10', 'L8', 'L5', 'L4', 'L1', 'L2', 'L3', 'L11']
Generation 40: 330
Current Best Score:  209.986343383789